In [68]:
import pandas as pd
import numpy as np 
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
from pyspark.sql import functions as F


from env import user, password, host

In [27]:
#create enviroment
spark = SparkSession.builder.getOrCreate()
spark

#### These exercises use the cases, dept, and source tables from the 311_data on the Codeup MySQL server.


In [28]:
def get_db_url(db):
    '''input df and output sql connection string'''
    return (f'mysql+pymysql://{user}:{password}@{host}/{db}')


In [ ]:


def get_311_data(file_name, query) -> pd.DataFrame: # add to wrangle file 
    if os.path.isfile(file_name):
        return pd.read_csv(file_name)
    connection = url
    df = pd.read_sql(query, connection)
    df.to_csv(file_name, index=False)
    return df

In [29]:
#sql connection 
url = get_db_url('311_data')

#sql query
query1 = 'select * from cases'
query2 = 'select * from source'
query3 = 'select * from dept'

In [30]:
df = get_311_data('cases.csv', query1)
df = get_311_data('sources.csv', query2)
df = get_311_data('depts.csv', query3)

#### Read the cases, department, and source data into their own Spark dataframes.


In [38]:
cases_data = pd.read_csv('cases.csv')
cases = spark.createDataFrame(cases_data)
cases.show(3, vertical=True, truncate=False)

/Users/ramirolopez/anaconda3/envs/codeup/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:
23/10/26 14:22:20 WARN TaskSetManager: Stage 13 contains a task of very large size (11384 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 1/1/18 0:42                           
 case_closed_date     | 1/1/18 12:29                          
 SLA_due_date         | 9/26/20 0:42                          
 case_late            | NO                                    
 num_days_late        | -998.5087616                          
 case_closed          | YES                                   
 dept_division        | Field Operations                      
 service_request_type | Stray Animal                          
 SLA_days             | 999.0                                 
 case_status          | Closed                                
 source_id            | svcCRMLS                              
 request_address      | 2315  EL PASO ST, San Antonio, 78207  
 council_district     | 5                                     
-RECORD 1----------------------------------------------

23/10/26 14:22:24 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 13 (TID 13): Attempting to kill Python Worker


In [37]:
dept_data = pd.read_csv('depts.csv')
dept = spark.createDataFrame(dept_data)
dept.show(3, vertical=True, truncate=False)

/Users/ramirolopez/anaconda3/envs/codeup/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


-RECORD 0----------------------------------------
 dept_division          | 311 Call Center        
 dept_name              | Customer Service       
 standardized_dept_name | Customer Service       
 dept_subject_to_SLA    | YES                    
-RECORD 1----------------------------------------
 dept_division          | Brush                  
 dept_name              | Solid Waste Management 
 standardized_dept_name | Solid Waste            
 dept_subject_to_SLA    | YES                    
-RECORD 2----------------------------------------
 dept_division          | Clean and Green        
 dept_name              | Parks and Recreation   
 standardized_dept_name | Parks & Recreation     
 dept_subject_to_SLA    | YES                    
only showing top 3 rows



In [80]:
sources_data = pd.read_csv('sources.csv')
sources = spark.createDataFrame(sources_data)
sources.show(3, vertical=True, truncate=False)

/Users/ramirolopez/anaconda3/envs/codeup/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


-RECORD 0---------------------------
 index           | 0                
 source_id       | 100137           
 source_username | Merlene Blodgett 
-RECORD 1---------------------------
 index           | 1                
 source_id       | 103582           
 source_username | Carmen Cura      
-RECORD 2---------------------------
 index           | 2                
 source_id       | 106463           
 source_username | Richard Sanchez  
only showing top 3 rows




#### Let's see how writing to the local disk works in Spark:



- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json


In [44]:
# to json
sources.write.json('data/sources_json', mode='overwrite')
dept.write.json('data/dept_json', mode='overwrite')
cases.write.json('data/cases_json', mode='overwrite')

23/10/26 14:35:05 WARN TaskSetManager: Stage 19 contains a task of very large size (11384 KiB). The maximum recommended task size is 1000 KiB.


In [45]:
# to csv
sources.write.csv('data/sources_csv', mode='overwrite')
dept.write.csv('data/dept_csv', mode='overwrite')
cases.write.csv('data/cases_csv', mode='overwrite')

23/10/26 14:35:18 WARN TaskSetManager: Stage 22 contains a task of very large size (11384 KiB). The maximum recommended task size is 1000 KiB.


- Inspect your folder structure. What do you notice?


- /Users/ramirolopez/codeup-data-science/spark-exercises/data/:
    - sources_csv
    - sources_json
    - dept_csv
    - dept_json
    - cases_csv
    - cases_json
    
========================
- all saved in a folder in a directory 

#### Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

- inspecting sources

In [81]:
# Looking at sources dtypes 
sources.dtypes

[('index', 'bigint'), ('source_id', 'string'), ('source_username', 'string')]

In [82]:
# looking at the values 
# I see that source_id in an int 
sources.select('source_id', 'source_username').show(1, vertical=True, truncate=False)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



In [83]:
# investigated the column sourse_id to verify it's an int 
unique_count = sources.groupBy('source_id').agg(F.countDistinct('source_id').alias('count'))
unique_count.show(3)

+---------+-----+
|source_id|count|
+---------+-----+
|   124405|    1|
|   119403|    1|
|   137943|    1|
+---------+-----+
only showing top 3 rows



In [84]:
total_unique_count = unique_count.count()
print(total_unique_count)

139


In [87]:
# changing the data type in place using cast
sources = sources.withColumn(
    'source_id', F.col('source_id').cast('int')
    )

In [88]:
# Verity 
sources.dtypes

[('index', 'bigint'), ('source_id', 'int'), ('source_username', 'string')]

- inspecting dept

In [90]:
# Looking at current data types and column names 
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [96]:
# Looks like dept_subject_to_SLA can be changed to a bool value
dept.show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



In [98]:
# investigated the column dept_subject_to_SLA to verify it can be a bool 
unique_count = dept.groupBy('dept_subject_to_SLA').agg(F.countDistinct('dept_subject_to_SLA').alias('count'))
unique_count.show()

+-------------------+-----+
|dept_subject_to_SLA|count|
+-------------------+-----+
|                YES|    1|
|                 NO|    1|
+-------------------+-----+



In [99]:
# only 2 values YES or NO 
total_unique_count = unique_count.count()
print(total_unique_count)

2


In [101]:
# Changing the data type in place using cast 
dept = dept.withColumn(
    'dept_subject_to_SLA', 
    F.when(F.col('dept_subject_to_SLA') == 'YES', True).otherwise(False)
    )

In [102]:
# Verify 
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'boolean')]

- inspecting cases

In [103]:
# Looking at data types and column names 
cases.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

In [108]:
# Looking at values in the columns 
cases.show(1, vertical=True, truncate=False)

23/10/26 15:42:06 WARN TaskSetManager: Stage 101 contains a task of very large size (11384 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616                         
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



23/10/26 15:42:10 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 101 (TID 237): Attempting to kill Python Worker


- takeaways: Change the following Dtypes 

case_opened_date -> datetime

case_closed_date -> datetime

SLA_due_date -> datetime

case_late -> bool

case_closed -> bool

In [109]:
# Changing the data type in place using cast 
cases = cases.withColumn('case_late', F.when(F.col('case_late') == 'YES', True).otherwise(False))
cases = cases.withColumn('case_closed', F.when(F.col('case_closed') == 'YES', True).otherwise(False))

In [110]:
cases.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

In [112]:
# looking at the date time format 
# for all 3 columns 
cases.select('case_opened_date', 'case_closed_date', 'SLA_due_date').show(1, vertical=True, truncate=False)

23/10/26 15:49:34 WARN TaskSetManager: Stage 103 contains a task of very large size (11384 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0------------------------
 case_opened_date | 1/1/18 0:42  
 case_closed_date | 1/1/18 12:29 
 SLA_due_date     | 9/26/20 0:42 
only showing top 1 row



23/10/26 15:49:38 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 103 (TID 239): Attempting to kill Python Worker


In [113]:
# changing the data types in place 
cases = cases.withColumn('case_opened_date', F.to_timestamp('case_opened_date', 'M/d/yy H:mm'))
cases = cases.withColumn('case_closed_date', F.to_timestamp('case_closed_date', 'M/d/yy H:mm'))
cases = cases.withColumn('SLA_due_date', F.to_timestamp('SLA_due_date', 'M/d/yy H:mm'))

In [114]:
cases.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

### Explore 

- How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?


In [125]:
# View the columns 
cases.show(1)

23/10/26 16:16:55 WARN TaskSetManager: Stage 112 contains a task of very large size (11384 KiB). The maximum recommended task size is 1000 KiB.


+----------+-------------------+-------------------+-------------------+---------+-------------+-----------+----------------+--------------------+--------+-----------+---------+--------------------+----------------+
|   case_id|   case_opened_date|   case_closed_date|       SLA_due_date|case_late|num_days_late|case_closed|   dept_division|service_request_type|SLA_days|case_status|source_id|     request_address|council_district|
+----------+-------------------+-------------------+-------------------+---------+-------------+-----------+----------------+--------------------+--------+-----------+---------+--------------------+----------------+
|1014127332|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|    false| -998.5087616|       true|Field Operations|        Stray Animal|   999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
+----------+-------------------+-------------------+-------------------+---------+-------------+-----------+----------------+-----------

23/10/26 16:17:00 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 112 (TID 283): Attempting to kill Python Worker


In [137]:
# filtered the dataframe using case_closed to only show cases still open 
# ordered by num_days_late to see the highest number of days  
cases.filter((cases.case_closed == 'false')).orderBy(F.col('num_days_late').asc()).show(1)

23/10/26 16:51:20 WARN TaskSetManager: Stage 126 contains a task of very large size (11384 KiB). The maximum recommended task size is 1000 KiB.


+----------+-------------------+----------------+-------------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+------------+--------------------+----------------+
|   case_id|   case_opened_date|case_closed_date|       SLA_due_date|case_late|num_days_late|case_closed|dept_division|service_request_type|SLA_days|case_status|   source_id|     request_address|council_district|
+----------+-------------------+----------------+-------------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+------------+--------------------+----------------+
|1014727115|2018-07-27 15:07:00|            NULL|2022-05-27 15:07:00|    false| -1387.630417|      false|      Signals|Signal Timing Mod...|  1400.0|       Open|CRM_Listener|SB IH 35 and THEO...|               5|
+----------+-------------------+----------------+-------------------+---------+-------------+-----------+-------------+--------------------+--------

23/10/26 18:59:53 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 501315 ms exceeds timeout 120000 ms
23/10/26 18:59:53 WARN SparkContext: Killing executors is not supported by current scheduler.
23/10/26 19:15:52 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [133]:
# Shows the highest number of days late 
cases.filter((cases.case_closed == 'false') & (cases.select('num_days_late').orderBy(F.col('num_days_late').asc()).select('num_days_late'))).show()

AttributeError: 'DataFrame' object has no attribute '_get_object_id'

How many Stray Animal cases are there?


How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?



Convert the council_district column to a string column.



Extract the year from the case_closed_date column.
Convert num_days_late from days to hours in a new column num_hours_late.



Join the cases data with the source and department data.



Are there any cases that do not have a request source?



What are the top 10 service request types in terms of number of requests?



What are the top 10 service request types in terms of average days late?


Does number of days late depend on department? (Answer without the use of a stats test)


How do number of days late depend on department and request type?